In [2]:
import os
import numpy as np
import imageio
from matplotlib import pyplot as plt
from utils import *

In [3]:
tMax = 60
stock = 340
strikeMin = stock*0.93
strikeMax = stock*1.07

###
n = int((strikeMax - strikeMin)/0.01 + 1)
strike = np.linspace(strikeMin, strikeMax, n)

### t days until expiration
valueT = np.zeros((tMax+1, n))
deltaT = np.zeros((tMax+1, n))
deltaFactorT = np.zeros((tMax+1, n))
for t in range(tMax+1):
    value = BlackScholes(strike, stock, t)
    valueT[t] = value
    delta = Greeks(strike, stock, t, greek="delta")
    deltaT[t] = delta
    deltaFactorT[t] = stock*delta/value

### Normalizing Data

# normalizing stock value
moneyness = (stock - strike) / stock

/Users/peter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
/Users/peter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


In [3]:
# Generating images
files = []
for t in range(tMax+1)[::-1]:
    fig, ax1 = plt.subplots(figsize = [10, 8], tight_layout = True)
    xlabel = r"$\bf{Option\ Moneyness}$"
    xlabel += "\n"
    xlabel += r"$\rm{\left(\frac{stock\ price\ \minus\ strike\ price}{stock\ price}\right)}$"
    plt.xlabel(xlabel, fontsize = 16)    

    title = r"$\bf{Delta\ Value\ of\ Call\ Option\ Contracts}$"
    title += "\n"
    title += r"$\rm{Days\ to\ Expiration:}$"
    title += " %2s" % t
    plt.title(title, family = "monospace", fontsize = 16)
    
    xMin, xMax = np.min(moneyness), np.max(moneyness)
    plt.xlim(xMin, xMax)
    ticks = plt.gca().get_xticks()
    ticks = ticks[(ticks <= xMax)&(ticks >= xMin)]
    xLabels = []
    for curLabel in ticks:
        newLabel = "{:.0f}%\n".format(curLabel*100)
        if curLabel == 0:
            newLabel += r"$\it{ATM}$"
        elif curLabel == ticks[0]:
            newLabel += r"$\it{OTM}$"
        elif curLabel == ticks[-1]:
            newLabel += r"$\it{ITM}$"
        xLabels.append(newLabel)
    plt.xticks(ticks, xLabels, fontsize = 14)
    plt.grid()
    
    ax1.tick_params(axis='y', labelsize = 14, labelcolor='r')
    ax1.set_yticks(np.linspace(0, 1, 6))
    ax1.plot(moneyness, deltaT[t], color = 'r', linewidth = 2.5)
    ylabel = r"$\bf{Delta}$"
    ylabel += "\n"
    ylabel += r"$\rm{(Change\ in\ option\ value\ due\ to\ \$1\ stock\ increase)}$"
    ax1.set_ylabel(ylabel, color = 'r', fontsize = 16)
    ax1.set_ylim(0, 1)
    
    ax2 = ax1.twinx()
    ax2.plot(moneyness, deltaFactorT[t], color = 'b', linewidth = 2.5)
    ax2.tick_params(axis='y', labelsize = 14, labelcolor='b')
    ax2.set_yticks(np.linspace(0, 100, 6))
    ylabel = r"$\bf{Leverage}$"
    ylabel += "\n"
    ylabel += r"$\rm{\left(Delta\times\frac{stock\ price}{option\ price}\right)}$"
    ax2.set_ylabel(ylabel, color = 'b', fontsize = 16)
    ax2.set_ylim(0, 100)
    
    fname = "delta_%02d.png" % t
    files.append(fname)
    plt.savefig(fname)
    plt.close()

In [4]:
### Making movie animation
images = []
for i in range(tMax+1):
    fname = files[i]
    # make first and last frames longer
    if i == 0 or i == tMax:
        for _ in range(9):
            images.append(imageio.imread(fname))
    #
    images.append(imageio.imread(fname))
imageio.mimsave('Delta.gif', images, duration=0.1)

### cleanup
for fname in files:
    os.remove(fname)